## PLUTO - PERCHWELL Building data merge

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
pluto = pd.read_csv("/Users/perchwellallusers/Downloads/pluto_23v1.csv")

/var/folders/9s/qkj1lg4d22j6vzbkv4zxvqmm0000gp/T/ipykernel_42762/1775343207.py:1: DtypeWarning: Columns (21,22,24,26,66) have mixed types. Specify dtype option on import or set low_memory=False.
  pluto = pd.read_csv("/Users/perchwellallusers/Downloads/pluto_23v1.csv")


In [ ]:
# First round - BBL
# Second round - Parcel id
# Third round - Address


In [58]:
import pandas as pd
import sqlalchemy as db
from sqlalchemy import create_engine, text
import pgspecial
import numpy as np
import recordlinkage
from sqlalchemy.orm import sessionmaker

In [27]:
# load sql_Magic
%load_ext sql

In [28]:
#connection string
%sql postgresql://teleport:@localhost:2024/perchwell

In [116]:

rowlist = []
# Create the SQLAlchemy engine
engine = create_engine('postgresql://teleport:@localhost:2024/perchwell')
currentrow = len(rowlist)
# Define the SQL query string
for ran in range(0,800000,100000):
    if len(rowlist) ==0:
        n = 0
    else:
        n = 100000
    query_string = f'''select *
        from buildings
        ---join with geographies
        inner join building_geographies on buildings.id = building_geographies.building_id
        ---filter for only NYC region code = 1278
        inner join (select * from geographies where geographies.id = 1278) as geo on geo.id = building_geographies.geography_id
        where source_id is not null
        LIMIT 100000 OFFSET {currentrow};
        '''
    try:
        # Execute the query
        with engine.connect() as conn:
            result = conn.execute(text(query_string))

            # Process the query result
            for row in result:
                # Access the row data
                rowlist.append(row)
        print(f'{len(rowlist)} rows appended')
    except:
        engine = create_engine('postgresql://teleport:@localhost:2024/perchwell')
        with engine.connect() as conn:
            result = conn.execute(text(query_string))
            for row in result:
                rowlist.append(row)
        print(f'{len(rowlist)} rows appended')


100000 rows appended
200000 rows appended
300000 rows appended
400000 rows appended
500000 rows appended
600000 rows appended
700000 rows appended
800000 rows appended


In [117]:
preso = pd.DataFrame(rowlist)

In [118]:
len(preso)

800000

In [119]:
preso['borough'] = preso['source_id'].astype(str).str[0]
preso['block'] = preso['source_id'].astype(str).str[1:6]
preso['lot'] = preso['source_id'].astype(str).str[6:10]

In [128]:
preso = preso[preso.source_id.notnull()]

In [121]:
preso[['borough','block','lot']] = preso[['borough','block','lot']].astype(int)

In [122]:
borough_mapping = {
    'MN': 1,
    'BX': 2,
    'BK': 3,
    'QN': 4,
    'SI': 5
}


In [123]:
pluto['borough'] = pluto['borough'].map(borough_mapping)

In [124]:
PWPLUTOmer = preso.merge(pluto, how = 'left', on = ['borough','block','lot'])

In [ ]:
PWPLUTOmer[PWPLUTOmer.borough.notnull() & PWPLUTOmer.bbl.notnull()].id.nunique()